# Augmenting a dataset for semantic segmentation

In this notebook, we illustrate how CLODSA can be employed to augment a dataset of images devoted to semantic segmentation. In particular, we use the dataset provided for the [2018 Data Science Bowl](https://www.kaggle.com/c/data-science-bowl-2018), that is divoted to find the nucle in divergent images to advance medical discovery - from now on we will call this dataset, the Nuclei dataset.

The Nuclei training dataset consists of 670 nucle images and their corresponding lbels. For illustration purposes, we take a subset of the Nucle dataset containing 100 images. Such a subset can be downloaded by executing the following command. 

We can check the amount of images in each one of the folders.

In [97]:
!pip3 install clodsa

In [116]:
print("Number of images")
!ls /home/ritesh/hdd1/vikram/ER_PR/extracted_patches/train/images/ | wc -l
print("Number of masks")
!ls /home/ritesh/hdd1/vikram/ER_PR/extracted_patches/train/labels/ | wc -l

Number of images
9092
Number of masks
9092


## Augmentation techniques

For this example, we consider the augmentation techniques applied in the work ["U-Net: Convolutional Networks for Biomedical Image Segmentation"](https://arxiv.org/abs/1505.04597), where they present a network and training strategy that relies on the strong use of data augmentation to use the available annotated samples more efficiently. Using such an approach, they won the [ISBI challenge for segmentation of neuronal structures in electron microscopic stacks](http://brainiac2.mit.edu/isbi_challenge/home). 

The augmentation techniques applied in that work are:
- Shifting.
- Rotation.
- Elastic deformations.

In addition, we also apply gamma correction. 

## Loading the necessary libraries

The first step in the pipeline consists in loading the necessary libraries to apply the data augmentation techniques in CLODSA.

In [117]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
import cv2
import numpy as np
%matplotlib inline

## Creating the augmentor object

As explained in the documentation of CLODSA, we need to specify some parameters for the augmentation process, and use them to create an augmentor object.  

_The kind of problem_. In this case, we are working in a semantic segmentation problem.

In [118]:
PROBLEM = "semantic_segmentation"

_The annotation mode_. The annotation is provided by the name of the folder containing the image. 

In [120]:
ANNOTATION_MODE = "folders"

_The input path_. The input path containing the images. 

In [121]:
# parent_path = '/home/ritesh/Desktop/Codes/ER/datasets/er_unet/train/'

In [122]:
INPUT_PATH = '/home/ritesh/hdd1/vikram/ER_PR/extracted_patches/train/'

_The generation mode_. In this case, linear, that is, all the augmentation techniques are applied to all the images of the original dataset. 

In [123]:
GENERATION_MODE = "linear"

_The output mode_. The generated images will be stored in a new folder called augmented_images_nuclei.  

In [124]:
OUTPUT_MODE = "folders"
OUTPUT_PATH= '/home/ritesh/hdd1/vikram/ER_PR/ep_aug_all/train/'
LABELS_EXTENSION = ".png"

Using the above information, we can create our augmentor object. 

In [125]:
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

## Adding the augmentation techniques

Now, we define the techniques that will be applied in our augmentation process and add them to our augmentor object. To illustrate the transformations, we will use the following image of the dataset. 



In [126]:
# img = cv2.imread("/home/ritesh/Desktop/Codes/ER/datasets/er_aug/test/images/image_3.png")
# label = cv2.imread( "/home/ritesh/Desktop/Codes/ER/datasets/er_aug/test/labels/image_3.png")
# # changing to the BGR format of OpenCV to RGB format for matplotlib
# plt.figure()
# plt.imshow(img[:,:,::-1])
# plt.title("Nuclei")
# plt.figure()
# plt.imshow(label[:,:,::-1]*60)
# plt.title("Label")
# dst = cv2.addWeighted(img,0.7,label,0.3,0)
# plt.figure()
# plt.imshow(dst[:,:,::-1])
# plt.title("Blending")

First of all, we must define a transformer generator.

In [127]:
transformer = transformerGenerator(PROBLEM)

_Rotations:_

In [128]:
for angle in [90]:
    rotate = createTechnique("rotate", {"angle" : angle})
    augmentor.addTransformer(transformer(rotate))

Showing the result of applying the transformation.

In [129]:
rotationGenerator = transformer(rotate)
# rotateImg,rotateLabel = rotationGenerator.transform(img,label)
# plt.figure()
# plt.imshow(rotateImg[:,:,::-1])
# plt.title("Nuclei")
# plt.figure()
# plt.imshow(rotateLabel[:,:,::-1]*60)
# plt.title("Label")
# dst = cv2.addWeighted(rotateImg,0.7,rotateLabel,0.3,0)
# plt.figure()
# plt.imshow(dst[:,:,::-1])
# plt.title("Blending")

**Flipping**

In [130]:
for flip_tp in [1]:
    flip = createTechnique("flip", {"flip" : flip_tp})
    augmentor.addTransformer(transformer(flip))
# hflip = createTechnique("flip",{"flip":1})
# augmentor.addTransformer(transformer(hflip))

In [131]:
flipGenerator = transformer(flip)
# flipImg,flipLabel = flipGenerator.transform(img,label)
# plt.figure()
# plt.imshow(flipImg[:,:,::-1])
# plt.title("Nuclei")
# plt.figure()
# plt.imshow(flipLabel[:,:,::-1]*60)
# plt.title("Label")
# dst = cv2.addWeighted(flipImg,0.7,flipLabel,0.3,0)
# plt.figure()
# plt.imshow(dst[:,:,::-1])
# plt.title("Blending")

#### Shifting

In [132]:
for translation in [1]:
    translation = createTechnique("translation", {"x":15,"y":-5})
    augmentor.addTransformer(transformer(translation))

Showing the result of applying the transformation.

In [133]:
translationGenerator = transformer(translation)
# translationImg,translationLabel = translationGenerator.transform(image,label)
# plt.figure()
# plt.imshow(translationImg[:,:,::-1])
# plt.title("Nuclei")
# plt.figure()
# plt.imshow(translationLabel[:,:,::-1])
# plt.title("Label")
# dst = cv2.addWeighted(translationImg,0.7,translationLabel,0.3,0)
# plt.figure()
# plt.imshow(dst[:,:,::-1])
# plt.title("Blending")

#### None
(to keep also the original image)

In [134]:
none = createTechnique("none",{})
augmentor.addTransformer(transformer(none))

## Applying the augmentation process

Finally, we apply the augmentation process (this might take some time depending on the number of images of the original dataset and the number of transformations that will be applied). 

In [135]:
augmentor.applyAugmentation()

Finally, we can check the amount of images in the output folder.

In [136]:
print("Number of images in augmented nuclei folder")
!ls /home/ritesh/hdd1/vikram/ER_PR/ep_aug_all/train/images/ | wc -l
print("Number of images in augmented nueclei label folder")
!ls /home/ritesh/hdd1/vikram/ER_PR/ep_aug_all/train/labels/ | wc -l

Number of images in augmented nuclei folder
36368
Number of images in augmented nueclei label folder
36368


**Reading npy files and saving them as images**

In [ ]:
import os
import skimage.io
import numpy as np
lbl_dir = '/home/arpit/Downloads/8_PR/PR_ER_Data_and_GT/Data_GT_ER_Patches/Smaller_Patches_ER/ER_Weak/256x256_90x90/Train/'
for fl in os.listdir(lbl_dir + 'labels_npy/'):
     img_npy = np.load(lbl_dir + 'labels_npy/' + fl).astype('uint8')
     #save_img(fl, img_npy)
     skimage.io.imsave(lbl_dir + 'labels/' + fl.split('.')[0] + '.png', img_npy)
     print(fl)
    

In [ ]:
# lbl_dir = '/media/siddarth/119940757D37B1B4/HER2_work/Patches/256x256_160x160/Train/'

# img_npy = skimage.io.imread(lbl_dir + 'labels_images/' + 'ROI_17_0_0_0__000.png')

# np.unique(img_npy)

# img_npy = np.load(lbl_dir + 'Labels/' + 'ROI_3_0_0_0__000.npy')#.astype('uint8')

# np.unique(img_npy)